In [1]:
##################################################################
#           《Python人工智能编程实践（2024年度版）》开源代码
#-----------------------------------------------------------------
#            @章节号：6.9.3.2（层标准化的TensorFlow实践）                                                                           
#            @作者：范淼、徐晟桐 
#            @购书链接：暂无
#            @电子邮箱：fm12@tsinghua.org.cn             
#            @官方交流QQ群号：561500762                        
##################################################################

In [2]:
from tensorflow.keras import models, layers, losses, optimizers, Model


#设定超参数。
HIDDEN_SIZE = 256 
NUM_CLASSES = 10
EPOCHS = 5
BATCH_SIZE = 64
LEARNING_RATE = 1e-3


class DNN_LN(Model):
    '''
    自定义带有层标准化功能的全连接神经网络类，继承自keras.Model。
    '''
    def __init__(self, hidden_size, num_classes):
        
        super(DNN_LN, self).__init__()
        
        self.l1 = layers.Dense(hidden_size, activation='relu')
        
        self.ln = layers.LayerNormalization()
    
        self.l2 = layers.Dense(num_classes)


    def call(self, input_tensor):
        
        out = self.l1(input_tensor)
        
        out = self.ln(out)
        
        out = self.l2(out)
        
        return out
    

#初始化带有层标准化功能的全连接神经网络。
model = DNN_LN(HIDDEN_SIZE, NUM_CLASSES)

#设定神经网络的损失函数、优化方式，以及评估方法。
model.compile(optimizer=optimizers.Adam(LEARNING_RATE), 
              loss=losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

In [3]:
import pandas as pd


#使用pandas，读取fashion_mnist的训练和测试数据文件。
train_data = pd.read_csv('./datasets/fashion_mnist/fashion_mnist_train.csv')
test_data = pd.read_csv('./datasets/fashion_mnist/fashion_mnist_test.csv')

#从训练数据中，拆解出训练特征和类别标签。
X_train = train_data[train_data.columns[1:]]
y_train = train_data['label']

#从测试数据中，拆解出测试特征和类别标签。
X_test = test_data[train_data.columns[1:]]
y_test = test_data['label']

In [4]:
from sklearn.preprocessing import StandardScaler


#初始化数据标准化处理器。
ss = StandardScaler()

#标准化训练数据特征。
X_train = ss.fit_transform(X_train)

#标准化测试数据特征。
X_test = ss.transform(X_test)

In [5]:
#使用fashion_mnist的训练集数据训练网络模型。
model.fit(X_train, y_train.values, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1)

Epoch 1/5
 49/938 [>.............................] - ETA: 0s - loss: 0.8119 - accuracy: 0.7152  

2023-04-11 19:35:41.472958: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


938/938 [==============================] - 1s 952us/step - loss: 0.4329 - accuracy: 0.8441
Epoch 2/5
938/938 [==============================] - 1s 942us/step - loss: 0.3217 - accuracy: 0.8812
Epoch 3/5
938/938 [==============================] - 1s 944us/step - loss: 0.2837 - accuracy: 0.8940
Epoch 4/5
938/938 [==============================] - 1s 941us/step - loss: 0.2560 - accuracy: 0.9054
Epoch 5/5
938/938 [==============================] - 1s 922us/step - loss: 0.2332 - accuracy: 0.9134


In [6]:
#使用fashion_mnist的测试集数据评估网络模型的效果。
result = model.evaluate(X_test, y_test.values, verbose=0)

print('带有层标准化功能的全连接神经网络（TensorFlow版本）在fashion_mnist测试集上的准确率为: %.2f%%。' %(result[1] * 100))

带有层标准化功能的全连接神经网络（TensorFlow版本）在fashion_mnist测试集上的准确率为: 89.08%。
